In [63]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import selenium
import pandas as pd
import time

In [2]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204%3A19&blank_scope=Latest"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'html.parser')

In [ ]:
#print(soup.prettify())

In [3]:
container_box = soup.find_all('div', class_='image_and_description_container')

In [4]:
#link = soup.find_all('li', class_='slide')
title = container_box[0].find_all('img')[1]['alt']
para = container_box[0].find_all('a')[0].text.strip()

# JPL Mars Space Image

In [5]:
# Create an executable path, initialize chrome driver
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path)
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
browser.click_link_by_id('full_image')
moreButton = browser.links.find_by_partial_text('more info')
moreButton.click()
featured_url = browser.html
featured_image = BeautifulSoup(featured_url, "html.parser")
jpl_image_url = featured_image.select_one("figure.lede a img").get("src")
full_feature_url = f"https://www.jpl.nasa.gov/{jpl_image_url}"
print(full_feature_url)
browser.quit()

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA20263_hires.jpg


# Mars Weather

In [6]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path)
mars_twit_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_twit_url)
mar_twit_url = browser.html
mar_twit = BeautifulSoup(mar_twit_url,"html.parser")

In [7]:
mar_twit_url = browser.html
mar_twit = BeautifulSoup(mar_twit_url,"html.parser")
twits = mar_twit.find_all("div", class_="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
mars_weather = twits[0].span.text
print(mars_weather)
browser.quit()

InSight sol 553 (2020-06-16) low -90.8ºC (-131.4ºF) high 0.0ºC (32.1ºF)
winds from the SW at 5.0 m/s (11.1 mph) gusting to 17.1 m/s (38.3 mph)
pressure at 7.50 hPa


# Mars Facts

In [10]:
mar_fact_url = "https://space-facts.com/mars/"
mar_fact = pd.read_html(mar_fact_url)[0]
mar_fact

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


# Mars Hemispheres

In [11]:
mar_hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
request_hemi = requests.get(mar_hemi_url)
hemi_soup = BeautifulSoup(request_hemi.text, 'html.parser')

In [51]:
hemi_names = hemi_soup.find_all('h3')
hemi_links = hemi_soup.find_all('a',class_='itemLink product-item')
mocklist = [hemi_names,hemi_links]
#hemi_name[0].text.replace("Enhanced", "")

In [64]:
hemisphere_image_urls = []
mini_url = "https://astrogeology.usgs.gov"
for name,links in zip(hemi_names, hemi_links):
    hemi_name = name.text.replace(" Enhanced","")
    full_url = mini_url + links['href']
    full_image_request = requests.get(full_url)
    full_image_soup = BeautifulSoup(full_image_request.text, 'html.parser')
    full_image_link = full_image_soup.find('img', class_="wide-image")
    full_image_total_url = mini_url + full_image_link['src']
    hemisphere_image_urls.append({'title':hemi_name,"img_url":full_image_total_url})
    time.sleep(1)
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]